## 문제 1. 
- 다음은 E-Commerce Shipping 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 고객이 주문한 물품의 정시 도착 여부를 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 예측 양식 : 제출한 예측값의 ROC_AUC 점수 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [1]:
import pandas as pd 
pd.DataFrame({
    "ID"  : [0, 1, 2], 
    "y_pred" : [0.2543, 0.1324, 0.5892]
})

,ID,y_pred
0,0,0.2543
1,1,0.1324
2,2,0.5892


## 데이터 불러오기

In [2]:
import pandas as pd 
X_test = pd.read_csv("data/2회/204_x_test.csv")
X_train = pd.read_csv("data/2회/204_x_train.csv")
y_train = pd.read_csv("data/2회/204_y_train.csv")

In [3]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Reached.on.Time_Y.N  6599 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB


## 데이터 정보

In [4]:
import pandas as pd 
pd.DataFrame({
    "변수" : list(X_train.columns) + ["Reached.on.Time_Y.N"], 
    "설명" : [
        "고객 ID 번호", 
        "창고의 블록 단위 구역(A, B, C, D, F)", 
        "제품 배송 방법", 
        "문의 전화 수", 
        "고객의 등급(1: 가장 낮음, 5: 가장 높음)", 
        "제품의 비용(달러 기준)",
        "사전 구매 수량", 
        "제품의 중요도", 
        "성별(F: 여성, M: 남성)", 
        "할인혜택", 
        "그램 단위 무게", 
        "정시 도착 여부(1: 도착하지 않음, 0: 정시 도착)"
    ]
})

,변수,설명
0,ID,고객 ID 번호
1,Warehouse_block,"창고의 블록 단위 구역(A, B, C, D, F)"
2,Mode_of_Shipment,제품 배송 방법
3,Customer_care_calls,문의 전화 수
4,Customer_rating,"고객의 등급(1: 가장 낮음, 5: 가장 높음)"
5,Cost_of_the_Product,제품의 비용(달러 기준)
6,Prior_purchases,사전 구매 수량
7,Product_importance,제품의 중요도
8,Gender,"성별(F: 여성, M: 남성)"
9,Discount_offered,할인혜택


In [20]:
import pandas as pd 
X_test = pd.read_csv("data/2회/204_x_test.csv")
X_train = pd.read_csv("data/2회/204_x_train.csv")
y_train = pd.read_csv("data/2회/204_y_train.csv")
# 코드 작성, 암기를 못하면 끝!
# 데이터 확인
#print(X_train.head(1))
#print(y_train.head(2))

# 결측치 확인
# print(X_train.isnull().sum()) #결측치 없음
# print(X_test.isnull().sum()) #결측치 없음

# 불필요한 컬럼 삭제
X_train_id = X_train.pop('ID') # X_train_id는 저장, X_train.pop('ID') # ID 컬럼은 제거 된 채로 DataFrame 저장
X_test_id = X_test.pop('ID')
# print(y_train['Reached.on.Time_Y.N'].value_counts()) # 비율의 차이가 나오니까, stratify 옵션 사용해야 하는 것을 깨닮음

#print(X_train.describe()) # 피처 엔지니어링은 RobustScaler 사용 해야지 결정.

# pipeline 사용 시, 사용하고자 하는 컬럼명을 추출하는 것이 중요
# 숫자형 컬럼 추출
import numpy as np 
numbers_df = X_train.select_dtypes(include=np.number)
# 문자형 컬럼 추출
object_df = X_train.select_dtypes(exclude=np.number) # OrdinalEncoder, One-Hot Encoder분리

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size =0.25, stratify = y_train['Reached.on.Time_Y.N'], random_state=42)

In [27]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import numpy as np

# RandomizedSearchCV() 에 대입
param_grid = {
    'classifier__n_estimators' : np.arange(30, 1000, 100), # 필수 
    'classifier__max_depth' : [None] + list(np.arange(5, 30, 5)), # 필수
    'classifier__min_samples_split': [2, 5, 10], # 옵션
    'classifier__min_samples_leaf': [1, 2, 4], # 옵션 
    'classifier__max_features': ['auto', 'sqrt', 'log2'], # 옵션
} 

# ColumnTransformer
column_transformer = ColumnTransformer([
    ("scaler", RobustScaler(), numbers_df.columns), # scaler
    ("ohc_encoder", OneHotEncoder(), ['Warehouse_block', 'Mode_of_Shipment', 'Gender']), # 명목척도 처리
    ("pd_imp_encoder", OrdinalEncoder(categories=[['low', 'medium', 'high']]), ['Product_importance']), # 서열척도 처리
], remainder = 'passthrough')

pipeline = Pipeline([
    ('preprocessor', column_transformer), 
    ('classifier', RandomForestClassifier(random_state=42))
])

# Grid Search 코드 스타일은 비슷(99% 비슷)
random_search = RandomizedSearchCV(
    estimator = pipeline, 
    param_distributions = param_grid, 
    n_iter = 3, 
    scoring = 'roc_auc', # 잘 모르겠다. 시험장에서는 디폴트 값으로 accuracy
    cv = 3, 
    verbose=3, 
    n_jobs = -1 # CPU cores 모두 사용하겠다는 뜻
)

# 모형학습
random_search.fit(X_tr, y_tr)
# 예측값 산출
predictions = random_search.predict_proba(X_te)

# 모형평가
roc_auc_score(y_te, predictions[:, 1])

Fitting 3 folds for each of 3 candidates, totalling 9 fits


C:\Users\chlwl\anaconda3\envs\ml_j2nhyeok\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
6 fits failed out of a total of 9.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\chlwl\anaconda3\envs\ml_j2nhyeok\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\chlwl\anaconda3\envs\ml_j2nhyeok\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\chlwl\anaconda3\envs\ml_j2nhyeok\lib\site-packages\sklearn\pipeline.py", line 420, in fit
    self._fina

0.7343200641196901